# Group 3 - Insurance Claim Data Analysis

## I. INTRODUCTION

## 1.1 Dataset

This is a relational dataset that comprehensively describes the various entities involved in the insurance claim process of an insurance company. The dataset is structured into **10 interconnected sheets**, each containing specific types of information:

- **Coverages**:  
  Details of the items covered by the insurance, along with their codes.  
  For example, types of covered expenses such as **Health**, **Life**, and **Other Life**.

- **Products**:  
  Information about different insurance products, such as **mortgage insurance**, **motor insurance**, **life insurance**, and **outpatient care**.

- **Participants**:  
  Personal and demographic information about customers participating in insurance policies.

- **Brokers**:  
  Details about brokers who distribute and sell insurance products.

- **Policies**:  
  Information related to insurance contracts, including **start date**, **cancellation date**, **renewal month**, and more.

- **Regions**:  
  Demographic and income-related data categorized by geographic regions.

- **State Regions**:  
  Regional codes mapped to corresponding U.S. states.

- **Claims Announcements**:  
  Records of claim announcements made by customers when initiating the claims process.

- **Claims Payments**:  
  Records of individual claim payment transactions made to customers.

- **Claim Reserves**:  
  Information on reserved funds for claims that have been initiated but not yet fully settled.


## II. DATA CLEANING

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
file_path = r"D:\Study\University\Năm 3_Kỳ 2\Big Data\Assignment\Insurance-Claims_Data_Model.xlsx"

#### Checking for Nan values and duplicates in each sheet of the dataset

In [3]:
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

# Dictionary for storing DataFrames for each sheet
dfs = {}

# Function to analyze a each sheet
def analyze_dataframe(name, df):
    print(f"\n--- Sheet: {name} ---")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    
    # Checking for missing values
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print("🔻 N/A values in columns:")
        print(missing)
    else:
        print("✅ No N/A values in the sheet.")
    
    # Checking for duplicate rows
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        print(f"🔻There is {dup_count} duplicate rows.")
    else:
        print("✅ There is no duplicate rows.")
    
    # Decriptive statistics
    print("📌 Data types for each column:")
    print(df.dtypes)

# Loop through each sheet and analyze it
for sheet in sheet_names:
    df = xls.parse(sheet)
    dfs[sheet] = df
    analyze_dataframe(sheet, df)


--- Sheet: Coverages ---
Shape: 177 rows × 8 columns
🔻 N/A values in columns:
RenewalType    109
Room           115
dtype: int64
✅ There is no duplicate rows.
📌 Data types for each column:
CoverID                 int64
CoverCode              object
RenewalType            object
Room                   object
Participation           int64
ProductCategory        object
PremiumMode            object
ProductDistribution    object
dtype: object

--- Sheet: Products ---
Shape: 77 rows × 4 columns
✅ No N/A values in the sheet.
✅ There is no duplicate rows.
📌 Data types for each column:
ProductID              int64
ProductCategory       object
ProductSubCategory    object
Product               object
dtype: object

--- Sheet: Participants ---
Shape: 12305 rows × 9 columns
🔻 N/A values in columns:
FirstName        183
BirthDate        184
Gender           184
MaritalStatus    184
dtype: int64
✅ There is no duplicate rows.
📌 Data types for each column:
ParticipantID               int64
Participa

#### ***Conclusion about the results***:
- Beside, the data type of BrokerID is float64, suggesting that the data is not clean => Cleaning the data then converting back to int64

![Cleaning Process](cleaning_process.jpg)

In [4]:
# Data cleaning function for each sheet
def clean_coverages(df):
    # Replace missing values in 'RenewalType' with 'Unknown' and drop the 'Room' column
    df['RenewalType'] = df['RenewalType'].fillna('Unknown')
    df.drop(columns=['Room'], inplace=True)
    return df

def clean_participants(df):
    # Remove rows with missing 'FirstName' or 'BirthDate'
    df.dropna(subset=['FirstName', 'BirthDate'], inplace=True)
    # Fill missing values in 'Gender' and 'MaritalStatus' with the most common value (mode)
    df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])  # Direct assignment instead of inplace
    df['MaritalStatus'] = df['MaritalStatus'].fillna(df['MaritalStatus'].mode()[0])  # Direct assignment instead of inplace
    return df

def clean_brokers(df):
    # Drop rows where all of the critical columns are missing
    critical_columns = ['BrokerID', 'BrokerCode', 'BrokerFullName', 'DistributionNetwork', 'CommissionScheme']
    df.dropna(subset=critical_columns, how='all', inplace=True)
    # Convert 'BrokerID' to integer
    df['BrokerID'] = df['BrokerID'].astype(int)
    return df

def clean_policies(df):
    # Replace 'NaN' in 'CancelationDate' with '2099-12-31' and remove rows with missing 'InsuredID'
    df['CancelationDate'] = pd.to_datetime(df['CancelationDate'], errors='coerce')
    df['CancelationDate'] = df['CancelationDate'].fillna(pd.to_datetime('2099-12-31'))  # Fill NaN with '2099-12-31'
    df.dropna(subset=['InsuredID'], inplace=True)
    return df

def clean_claims_announcements(df):
    # Replace 'NaN' in 'ClosingDate' with '2099-12-31' and remove rows with missing 'InsuredID'
    df['ClosingDate'] = pd.to_datetime(df['ClosingDate'], errors='coerce')
    df['ClosingDate'] = df['ClosingDate'].fillna(pd.to_datetime('2099-12-31'))  # Fill NaN with '2099-12-31'
    df.dropna(subset=['InsuredID'], inplace=True)
    return df

def clean_claims_payments(df):
    # Remove duplicate rows
    df.drop_duplicates(inplace=True)
    # Remove rows with missing 'PaymentDate' and 'InsuredID'
    df.dropna(subset=['PaymentDate', 'InsuredID'], inplace=True)
    # If only 'ClosingDate' is missing, fill it with the value from 'PaymentDate'
    df['ClosingDate'] = df['ClosingDate'].fillna(df['PaymentDate'])  # Direct assignment instead of inplace
    # Remove rows where both 'ClosingDate' and 'PaymentDate' are missing
    df.dropna(subset=['ClosingDate', 'PaymentDate'], how='all', inplace=True)
    return df

def clean_claims_reserves(df):
    # Replace 'NaN' in 'ClosingDate' with '2099-12-31' and remove rows with missing 'InsuredID'
    df['ClosingDate'] = pd.to_datetime(df['ClosingDate'], errors='coerce')
    df['ClosingDate'] = df['ClosingDate'].fillna(pd.to_datetime('2099-12-31'))  # Fill NaN with '2099-12-31'
    df.dropna(subset=['InsuredID'], inplace=True)
    return df


# Apply data cleaning for each sheet
cleaned_dfs = {}
cleaned_dfs['Coverages'] = clean_coverages(dfs['Coverages'])
cleaned_dfs['Participants'] = clean_participants(dfs['Participants'])
cleaned_dfs['Brokers'] = clean_brokers(dfs['Brokers'])
cleaned_dfs['Policies'] = clean_policies(dfs['Policies'])
cleaned_dfs['Claims Announcements'] = clean_claims_announcements(dfs['Claims Announcements'])
cleaned_dfs['Claims Payments'] = clean_claims_payments(dfs['Claims Payments'])
cleaned_dfs['Claims Reserves'] = clean_claims_reserves(dfs['Claims Reserves'])

# Save the cleaned data to a new file
output_path = r"D:\Study\University\Năm 3_Kỳ 2\Big Data\Assignment\Cleaned_Insurance_Claims_Data.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in cleaned_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("The data has been cleaned and successfully saved to the new file.")

The data has been cleaned and successfully saved to the new file.


In [5]:
# Check the dataset again
for sheet_name, df in cleaned_dfs.items():
    analyze_dataframe(sheet_name, df)


--- Sheet: Coverages ---
Shape: 177 rows × 7 columns
✅ No N/A values in the sheet.
✅ There is no duplicate rows.
📌 Data types for each column:
CoverID                 int64
CoverCode              object
RenewalType            object
Participation           int64
ProductCategory        object
PremiumMode            object
ProductDistribution    object
dtype: object

--- Sheet: Participants ---
Shape: 12121 rows × 9 columns
✅ No N/A values in the sheet.
✅ There is no duplicate rows.
📌 Data types for each column:
ParticipantID               int64
ParticipantCode            object
LastName                   object
FirstName                  object
BirthDate          datetime64[ns]
Gender                     object
ParticipantType            object
RegionID                    int64
MaritalStatus              object
dtype: object

--- Sheet: Brokers ---
Shape: 707 rows × 6 columns
✅ No N/A values in the sheet.
✅ There is no duplicate rows.
📌 Data types for each column:
BrokerID             